In [1]:
import mlflow
import time

In [2]:
mlflow_uri = "http://127.0.0.1:8888/"
mlflow.set_tracking_uri(mlflow_uri)
time.sleep(5)
#mlflow.set_experiment(mlflow_uri)
mlflow.set_experiment("v5_tf_mlflow_end_to_end")

<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1726977140826, experiment_id='2', last_update_time=1726977140826, lifecycle_stage='active', name='v5_tf_mlflow_end_to_end', tags={}>

In [3]:
import os
import tensorflow as tf
tf.config.list_physical_devices('GPU')
img_size = 150
batch_size = 16

In [4]:
root_path = r"/Users/tharhtet/Documents/github/ML-in-Prod-batch-1/5_DeepLearning/tf_best_practices/cats_and_dogs_filtered"

In [5]:
tf_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    rescale= 1/255.0

)

test_gen = tf_generator.flow_from_directory(
    root_path+"/test",
    target_size=(img_size, img_size),
    class_mode='binary', #'categorical',
    batch_size=batch_size,
    shuffle=True,
)


#By calling .repeat(), the dataset will loop indefinitely, providing data for all the epochs.

Found 248 images belonging to 2 classes.


In [11]:
for image,label in test_gen:
    print(len(image))
    test_img = image
    test_label = label
    break

16


In [7]:
import mlflow
logged_model = 'runs:/33c4ebd877844aa59d881ccf9f4954e1/cat_and_dog_model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

/opt/anaconda3/envs/dev_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
print(test_label)

[1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0.]


In [9]:
y_pred = loaded_model.predict(test_img)
print(y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
[[0.6535938 ]
 [0.78264064]
 [0.54850334]
 [0.5309534 ]
 [0.51534986]
 [0.6922578 ]
 [0.3688799 ]
 [0.57882786]
 [0.38975346]
 [0.598549  ]
 [0.50652456]
 [0.8094746 ]
 [0.6756455 ]
 [0.49007854]
 [0.5248701 ]
 [0.5752574 ]]


### Register the model

In [11]:
model_name = "ths_cat_and_dog_model"
model_uri = 'runs:/33c4ebd877844aa59d881ccf9f4954e1/cat_and_dog_model'
model_version = mlflow.register_model(model_uri,model_name)
print("model_version : ",model_version)

Registered model 'ths_cat_and_dog_model' already exists. Creating a new version of this model...
2024/09/22 11:23:42 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ths_cat_and_dog_model, version 3


model_version :  <ModelVersion: aliases=[], creation_timestamp=1726979022264, current_stage='None', description='', last_updated_timestamp=1726979022264, name='ths_cat_and_dog_model', run_id='33c4ebd877844aa59d881ccf9f4954e1', run_link='', source='mlflow-artifacts:/2/33c4ebd877844aa59d881ccf9f4954e1/artifacts/cat_and_dog_model', status='READY', status_message='', tags={}, user_id='', version='3'>


Created version '3' of model 'ths_cat_and_dog_model'.


In [12]:
# load model by tag
model_name = "ths_cat_and_dog_model"
load_model_uri = f"models:/{model_name}@prod_test"
loaded_model = mlflow.pyfunc.load_model(load_model_uri)
y_pred = loaded_model.predict(test_img)
print(y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
[[0.37041312]
 [0.1400522 ]
 [0.30446422]
 [0.72236407]
 [0.5680767 ]
 [0.5107035 ]
 [0.27472338]
 [0.41546285]
 [0.50497156]
 [0.319856  ]
 [0.2560752 ]
 [0.4645527 ]
 [0.6102773 ]
 [0.5221177 ]
 [0.33477825]
 [0.60212994]]
